# 1. Singlevariable Linear regression

In [1]:
import torch

x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

W = torch.zeros(1)
b = torch.zeros(1)

### torch.optim 없이 구현

In [2]:
epochs = 1000
lr = 0.01
for epoch in range(epochs):
    hypothesis = x_train*W+b
    cost = torch.mean((hypothesis-y_train)**2)
    
    gradient_W = torch.sum((hypothesis-y_train)*x_train)
    gradient_b = torch.sum(hypothesis-y_train)
    
    if epoch%100==0:
        print(cost,W,b)
    W -= lr*gradient_W
    b -= lr*gradient_b

tensor(18.6667) tensor([0.]) tensor([0.])
tensor(0.0378) tensor([1.7740]) tensor([0.5137])
tensor(0.0184) tensor([1.8425]) tensor([0.3579])
tensor(0.0089) tensor([1.8903]) tensor([0.2494])
tensor(0.0043) tensor([1.9236]) tensor([0.1738])
tensor(0.0021) tensor([1.9467]) tensor([0.1211])
tensor(0.0010) tensor([1.9629]) tensor([0.0844])
tensor(0.0005) tensor([1.9741]) tensor([0.0588])
tensor(0.0002) tensor([1.9820]) tensor([0.0410])
tensor(0.0001) tensor([1.9874]) tensor([0.0285])


In [3]:
W,b

(tensor([1.9912]), tensor([0.0199]))

### torch.optim 사용하여 구현

In [4]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

W = torch.zeros(1,requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [5]:
optimizer = torch.optim.SGD([W,b],lr=0.01)
epochs = 10000

for epoch in range(epochs):
    hypothesis = x_train*W+b
    cost = torch.mean((hypothesis-y_train)**2)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() #gradient 계산
    optimizer.step() #optimizer 학습

In [6]:
print(f'W :{W[0]}, b :{b[0]}')

W :1.9999957084655762, b :8.701807018951513e-06


# 2. Multivariable Linear regression

> * n x 3 matrix
> * 그에 따른 가중치 W도 3x1 로 설정

In [36]:
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,90],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

W = torch.zeros((3,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = torch.optim.SGD([W,b],lr=1e-5)

In [39]:
epochs = 10000

for epoch in range(epochs):
    hypothesis = x_train@W+b
    cost = torch.mean((hypothesis-y_train)**2)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() #gradient 계산
    optimizer.step() #optimizer 학습

In [40]:
W.squeeze(),b,cost

(tensor([0.9329, 0.4657, 0.6123], grad_fn=<SqueezeBackward0>),
 tensor([0.0284], requires_grad=True),
 tensor(0.2271, grad_fn=<MeanBackward0>))

### nn.module을 이용해 구현
> * 기존 hypothesis와 loss를 미리 구현
> * 모델이 바뀔때마다 쉽게 수정 가능
> * 버그도 적음

In [41]:
import torch.nn as nn

class MultivariabelLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1) # linear는 3x1 을 가지는 weight
    def forward(self,x):
        return self.linear(x) # forward를 linear로 정의

In [42]:
model = MultivariabelLinearRegressionModel()
optimizer = torch.optim.SGD([W,b],lr=1e-5)

In [43]:
for epoch in range(epochs):
    prediction = model(x_train) # forward 계산
    cost = nn.functional.mse_loss(prediction,y_train) # mse_loss 계산
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() #gradient 계산
    optimizer.step() #optimizer 학습

In [44]:
# parameter가 업데이트 됐는지 확인
list(model.parameters()),cost

([Parameter containing:
  tensor([[-0.4828, -0.4621,  0.0699]], requires_grad=True),
  Parameter containing:
  tensor([-0.3149], requires_grad=True)],
 tensor(61057.1484, grad_fn=<MseLossBackward0>))

# 3. Dataset 사용
> * Dataset을 사용하여 minibatch 적용하기

In [45]:
from torch.utils.data import Dataset,DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73,80,75],
                        [93,88,93],
                        [89,91,90],
                        [96,98,100],
                        [73,66,70]]
        self.y_data = [[152],[185],[180],[196],[142]]
        
    def __len__(self):
        return len(self.x_data)
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x,y

In [46]:
dataset = CustomDataset()
dataloader = DataLoader(dataset,batch_size=2,shuffle=True)

W = torch.zeros((3,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
model = MultivariabelLinearRegressionModel()
optimizer = torch.optim.SGD([W,b],lr=1e-5)

In [47]:
for epoch in range(epochs):
    for batch_idx,samples in enumerate(dataloader):
        x_train,y_train = samples
        
        prediction = model(x_train) # forward 계산
        cost = nn.functional.mse_loss(prediction,y_train) # mse_loss 계산

        optimizer.zero_grad() # gradient 초기화
        cost.backward() #gradient 계산
        optimizer.step() #optimizer 학습
        
        print(f'Epoch: {epoch} Batch: {batch_idx+1}/{len(dataloader)} Cost: {cost.item()} ')

Epoch: 0 Batch: 1/3 Cost: 18207.115234375 
Epoch: 0 Batch: 2/3 Cost: 18602.119140625 
Epoch: 0 Batch: 3/3 Cost: 20489.693359375 
Epoch: 1 Batch: 1/3 Cost: 22272.654296875 
Epoch: 1 Batch: 2/3 Cost: 17535.291015625 
Epoch: 1 Batch: 3/3 Cost: 14492.2734375 
Epoch: 2 Batch: 1/3 Cost: 18207.115234375 
Epoch: 2 Batch: 2/3 Cost: 16819.158203125 
Epoch: 2 Batch: 3/3 Cost: 24055.615234375 
Epoch: 3 Batch: 1/3 Cost: 21205.82421875 
Epoch: 3 Batch: 2/3 Cost: 18602.119140625 
Epoch: 3 Batch: 3/3 Cost: 14492.2734375 
Epoch: 4 Batch: 1/3 Cost: 22272.654296875 
Epoch: 4 Batch: 2/3 Cost: 18207.115234375 
Epoch: 4 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 5 Batch: 1/3 Cost: 22988.78515625 
Epoch: 5 Batch: 2/3 Cost: 13820.44921875 
Epoch: 5 Batch: 3/3 Cost: 20489.693359375 
Epoch: 6 Batch: 1/3 Cost: 16819.158203125 
Epoch: 6 Batch: 2/3 Cost: 19273.9453125 
Epoch: 6 Batch: 3/3 Cost: 21921.95703125 
Epoch: 7 Batch: 1/3 Cost: 21205.82421875 
Epoch: 7 Batch: 2/3 Cost: 18602.119140625 
Epoch: 7 Batch: 3/3 C

Epoch: 114 Batch: 1/3 Cost: 17535.291015625 
Epoch: 114 Batch: 2/3 Cost: 17490.984375 
Epoch: 114 Batch: 3/3 Cost: 24055.615234375 
Epoch: 115 Batch: 1/3 Cost: 21205.82421875 
Epoch: 115 Batch: 2/3 Cost: 19273.9453125 
Epoch: 115 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 116 Batch: 1/3 Cost: 18207.115234375 
Epoch: 116 Batch: 2/3 Cost: 16819.158203125 
Epoch: 116 Batch: 3/3 Cost: 24055.615234375 
Epoch: 117 Batch: 1/3 Cost: 19273.9453125 
Epoch: 117 Batch: 2/3 Cost: 16819.158203125 
Epoch: 117 Batch: 3/3 Cost: 21921.95703125 
Epoch: 118 Batch: 1/3 Cost: 18602.119140625 
Epoch: 118 Batch: 2/3 Cost: 21205.82421875 
Epoch: 118 Batch: 3/3 Cost: 14492.2734375 
Epoch: 119 Batch: 1/3 Cost: 17490.984375 
Epoch: 119 Batch: 2/3 Cost: 17535.291015625 
Epoch: 119 Batch: 3/3 Cost: 24055.615234375 
Epoch: 120 Batch: 1/3 Cost: 13820.44921875 
Epoch: 120 Batch: 2/3 Cost: 22272.654296875 
Epoch: 120 Batch: 3/3 Cost: 21921.95703125 
Epoch: 121 Batch: 1/3 Cost: 13820.44921875 
Epoch: 121 Batch: 2/3 Cost:

Epoch: 235 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 236 Batch: 1/3 Cost: 22272.654296875 
Epoch: 236 Batch: 2/3 Cost: 13820.44921875 
Epoch: 236 Batch: 3/3 Cost: 21921.95703125 
Epoch: 237 Batch: 1/3 Cost: 22272.654296875 
Epoch: 237 Batch: 2/3 Cost: 13820.44921875 
Epoch: 237 Batch: 3/3 Cost: 21921.95703125 
Epoch: 238 Batch: 1/3 Cost: 18602.119140625 
Epoch: 238 Batch: 2/3 Cost: 21205.82421875 
Epoch: 238 Batch: 3/3 Cost: 14492.2734375 
Epoch: 239 Batch: 1/3 Cost: 16819.158203125 
Epoch: 239 Batch: 2/3 Cost: 22988.78515625 
Epoch: 239 Batch: 3/3 Cost: 14492.2734375 
Epoch: 240 Batch: 1/3 Cost: 22988.78515625 
Epoch: 240 Batch: 2/3 Cost: 16819.158203125 
Epoch: 240 Batch: 3/3 Cost: 14492.2734375 
Epoch: 241 Batch: 1/3 Cost: 18602.119140625 
Epoch: 241 Batch: 2/3 Cost: 18207.115234375 
Epoch: 241 Batch: 3/3 Cost: 20489.693359375 
Epoch: 242 Batch: 1/3 Cost: 17535.291015625 
Epoch: 242 Batch: 2/3 Cost: 22272.654296875 
Epoch: 242 Batch: 3/3 Cost: 14492.2734375 
Epoch: 243 Batch: 1/3 Co

Epoch: 357 Batch: 1/3 Cost: 22272.654296875 
Epoch: 357 Batch: 2/3 Cost: 13820.44921875 
Epoch: 357 Batch: 3/3 Cost: 21921.95703125 
Epoch: 358 Batch: 1/3 Cost: 17490.984375 
Epoch: 358 Batch: 2/3 Cost: 18602.119140625 
Epoch: 358 Batch: 3/3 Cost: 21921.95703125 
Epoch: 359 Batch: 1/3 Cost: 17535.291015625 
Epoch: 359 Batch: 2/3 Cost: 19273.9453125 
Epoch: 359 Batch: 3/3 Cost: 20489.693359375 
Epoch: 360 Batch: 1/3 Cost: 22988.78515625 
Epoch: 360 Batch: 2/3 Cost: 17490.984375 
Epoch: 360 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 361 Batch: 1/3 Cost: 17535.291015625 
Epoch: 361 Batch: 2/3 Cost: 17490.984375 
Epoch: 361 Batch: 3/3 Cost: 24055.615234375 
Epoch: 362 Batch: 1/3 Cost: 21205.82421875 
Epoch: 362 Batch: 2/3 Cost: 13820.44921875 
Epoch: 362 Batch: 3/3 Cost: 24055.615234375 
Epoch: 363 Batch: 1/3 Cost: 21205.82421875 
Epoch: 363 Batch: 2/3 Cost: 13820.44921875 
Epoch: 363 Batch: 3/3 Cost: 24055.615234375 
Epoch: 364 Batch: 1/3 Cost: 16819.158203125 
Epoch: 364 Batch: 2/3 Cost: 

Epoch: 463 Batch: 2/3 Cost: 18602.119140625 
Epoch: 463 Batch: 3/3 Cost: 14492.2734375 
Epoch: 464 Batch: 1/3 Cost: 19273.9453125 
Epoch: 464 Batch: 2/3 Cost: 17535.291015625 
Epoch: 464 Batch: 3/3 Cost: 20489.693359375 
Epoch: 465 Batch: 1/3 Cost: 22988.78515625 
Epoch: 465 Batch: 2/3 Cost: 13820.44921875 
Epoch: 465 Batch: 3/3 Cost: 20489.693359375 
Epoch: 466 Batch: 1/3 Cost: 18602.119140625 
Epoch: 466 Batch: 2/3 Cost: 21205.82421875 
Epoch: 466 Batch: 3/3 Cost: 14492.2734375 
Epoch: 467 Batch: 1/3 Cost: 19273.9453125 
Epoch: 467 Batch: 2/3 Cost: 21205.82421875 
Epoch: 467 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 468 Batch: 1/3 Cost: 17535.291015625 
Epoch: 468 Batch: 2/3 Cost: 17490.984375 
Epoch: 468 Batch: 3/3 Cost: 24055.615234375 
Epoch: 469 Batch: 1/3 Cost: 17490.984375 
Epoch: 469 Batch: 2/3 Cost: 18602.119140625 
Epoch: 469 Batch: 3/3 Cost: 21921.95703125 
Epoch: 470 Batch: 1/3 Cost: 18602.119140625 
Epoch: 470 Batch: 2/3 Cost: 18207.115234375 
Epoch: 470 Batch: 3/3 Cost: 

...

Epoch: 9428 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9429 Batch: 1/3 Cost: 17490.984375 
Epoch: 9429 Batch: 2/3 Cost: 18602.119140625 
Epoch: 9429 Batch: 3/3 Cost: 21921.95703125 
Epoch: 9430 Batch: 1/3 Cost: 19273.9453125 
Epoch: 9430 Batch: 2/3 Cost: 21205.82421875 
Epoch: 9430 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9431 Batch: 1/3 Cost: 17535.291015625 
Epoch: 9431 Batch: 2/3 Cost: 22272.654296875 
Epoch: 9431 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9432 Batch: 1/3 Cost: 18602.119140625 
Epoch: 9432 Batch: 2/3 Cost: 17490.984375 
Epoch: 9432 Batch: 3/3 Cost: 21921.95703125 
Epoch: 9433 Batch: 1/3 Cost: 17490.984375 
Epoch: 9433 Batch: 2/3 Cost: 17535.291015625 
Epoch: 9433 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9434 Batch: 1/3 Cost: 17535.291015625 
Epoch: 9434 Batch: 2/3 Cost: 19273.9453125 
Epoch: 9434 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9435 Batch: 1/3 Cost: 22988.78515625 
Epoch: 9435 Batch: 2/3 Cost: 17490.984375 
Epoch: 9435 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9

Epoch: 9542 Batch: 1/3 Cost: 17535.291015625 
Epoch: 9542 Batch: 2/3 Cost: 19273.9453125 
Epoch: 9542 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9543 Batch: 1/3 Cost: 18602.119140625 
Epoch: 9543 Batch: 2/3 Cost: 21205.82421875 
Epoch: 9543 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9544 Batch: 1/3 Cost: 13820.44921875 
Epoch: 9544 Batch: 2/3 Cost: 22988.78515625 
Epoch: 9544 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9545 Batch: 1/3 Cost: 17490.984375 
Epoch: 9545 Batch: 2/3 Cost: 22988.78515625 
Epoch: 9545 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9546 Batch: 1/3 Cost: 22988.78515625 
Epoch: 9546 Batch: 2/3 Cost: 16819.158203125 
Epoch: 9546 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9547 Batch: 1/3 Cost: 18602.119140625 
Epoch: 9547 Batch: 2/3 Cost: 21205.82421875 
Epoch: 9547 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9548 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9548 Batch: 2/3 Cost: 13820.44921875 
Epoch: 9548 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9549 Batch: 1/3 Cost: 18207.115234375 
Epoch:

Epoch: 9659 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9659 Batch: 2/3 Cost: 19273.9453125 
Epoch: 9659 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9660 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9660 Batch: 2/3 Cost: 13820.44921875 
Epoch: 9660 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9661 Batch: 1/3 Cost: 17535.291015625 
Epoch: 9661 Batch: 2/3 Cost: 17490.984375 
Epoch: 9661 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9662 Batch: 1/3 Cost: 18602.119140625 
Epoch: 9662 Batch: 2/3 Cost: 21205.82421875 
Epoch: 9662 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9663 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9663 Batch: 2/3 Cost: 18602.119140625 
Epoch: 9663 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9664 Batch: 1/3 Cost: 18207.115234375 
Epoch: 9664 Batch: 2/3 Cost: 16819.158203125 
Epoch: 9664 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9665 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9665 Batch: 2/3 Cost: 18602.119140625 
Epoch: 9665 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9666 Batch: 1/3 Cost: 16819.158203125 
Epoc

Epoch: 9780 Batch: 2/3 Cost: 22988.78515625 
Epoch: 9780 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9781 Batch: 1/3 Cost: 22988.78515625 
Epoch: 9781 Batch: 2/3 Cost: 16819.158203125 
Epoch: 9781 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9782 Batch: 1/3 Cost: 22988.78515625 
Epoch: 9782 Batch: 2/3 Cost: 16819.158203125 
Epoch: 9782 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9783 Batch: 1/3 Cost: 22272.654296875 
Epoch: 9783 Batch: 2/3 Cost: 17535.291015625 
Epoch: 9783 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9784 Batch: 1/3 Cost: 16819.158203125 
Epoch: 9784 Batch: 2/3 Cost: 22988.78515625 
Epoch: 9784 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9785 Batch: 1/3 Cost: 16819.158203125 
Epoch: 9785 Batch: 2/3 Cost: 18207.115234375 
Epoch: 9785 Batch: 3/3 Cost: 24055.615234375 
Epoch: 9786 Batch: 1/3 Cost: 22272.654296875 
Epoch: 9786 Batch: 2/3 Cost: 18207.115234375 
Epoch: 9786 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9787 Batch: 1/3 Cost: 18207.115234375 
Epoch: 9787 Batch: 2/3 Cost: 22272.654296875 

Epoch: 9902 Batch: 1/3 Cost: 17535.291015625 
Epoch: 9902 Batch: 2/3 Cost: 19273.9453125 
Epoch: 9902 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9903 Batch: 1/3 Cost: 13820.44921875 
Epoch: 9903 Batch: 2/3 Cost: 22272.654296875 
Epoch: 9903 Batch: 3/3 Cost: 21921.95703125 
Epoch: 9904 Batch: 1/3 Cost: 21205.82421875 
Epoch: 9904 Batch: 2/3 Cost: 18602.119140625 
Epoch: 9904 Batch: 3/3 Cost: 14492.2734375 
Epoch: 9905 Batch: 1/3 Cost: 22988.78515625 
Epoch: 9905 Batch: 2/3 Cost: 17490.984375 
Epoch: 9905 Batch: 3/3 Cost: 13148.6240234375 
Epoch: 9906 Batch: 1/3 Cost: 16819.158203125 
Epoch: 9906 Batch: 2/3 Cost: 19273.9453125 
Epoch: 9906 Batch: 3/3 Cost: 21921.95703125 
Epoch: 9907 Batch: 1/3 Cost: 19273.9453125 
Epoch: 9907 Batch: 2/3 Cost: 17535.291015625 
Epoch: 9907 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9908 Batch: 1/3 Cost: 19273.9453125 
Epoch: 9908 Batch: 2/3 Cost: 17535.291015625 
Epoch: 9908 Batch: 3/3 Cost: 20489.693359375 
Epoch: 9909 Batch: 1/3 Cost: 22988.78515625 
Epoch:

In [48]:
list(model.parameters()),cost

([Parameter containing:
  tensor([[-0.1643,  0.1606,  0.4069]], requires_grad=True),
  Parameter containing:
  tensor([0.2417], requires_grad=True)],
 tensor(21921.9570, grad_fn=<MseLossBackward0>))